1.Scrapping data from wbesite

1. Importing libraries essetial to scrapping

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

2. Pointing url to fetch data from

In [2]:
url = 'https://www.pig333.com/markets_and_prices/'
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

3. Locating tags where the data in relevant fetching into a dataframe

In [3]:
markets = []
listings = soup.find('div', {'class': 'contingut'}).find_all('div', {'class': 'preu_mercat'})

4. Fetching data from located tags in the HTML into a proper dataframe.

In [4]:
for item in listings:
    market = {}

    market['pais'] = item.find('a', {'class': 'titol'}).getText()
    market['data'] = item.select('.col-titol > span.data')[0].getText()
    market['preco'] = item.find('span', {'class': 'preu'}).getText()
    market['moeda'] = item.find('span', {'class': 'moneda'}).getText()
    market['unidade'] = item.find('span', {'class': 'unitats'}).getText()
    market['referencia'] = item.find('span', {'class': 'data fs9'}).getText()
    market['variacao'] = item.find('span', {'class': 'perc'}).getText()

    markets.append(market)
  

pricing = pd.DataFrame(markets)
pricing

,pais,data,preco,moeda,unidade,referencia,variacao
0,Belgium - Danis,30 Jul,0.960,EUR,kg,Live,2.1%
1,Denmark,30 Jul,10.50,DKK,kg,Carcass 60%,0%
2,France - MPB,3 Aug,1.293,EUR,kg,Carcass 56 TMV,0%
3,Germany,29 Jul,1.470,EUR,kg,Carcass 57%,0%
4,Italy,30 Jul,1.275,EUR,kg,160/176 kg Live,3.2%
5,Netherlands - Beursprijs,31 Jul,1.050,EUR,kg,Live,0%
6,Poland,24 Jul,6.261,PLN,kg,Carcass 57%,6.6%
7,Romania,24 Jul,536.79,RON,100kg,Live,2.0%
8,Russia,22 Jul,120.40,RUB,kg,Live,2.6%
9,Spain - Lleida,30 Jul,1.300,EUR,kg,Live,0%


In [5]:
##4.1 exporting prices for swine into a CSV file

In [6]:
import datetime

current_date = datetime.datetime.now()
filename = str(current_date.day)+str(current_date.month)+str(current_date.year)
pricing.to_csv(str('/Users/lstorino/PythonTests/scrapping/Pig333/data/pricetoday'+filename+ '.csv'))
#pricing.to_csv('/Users/lstorino/PythonTests/scrapping/Pig333/data/pricetoday.csv')

5. Create Unit equalizing column

In [7]:
type(pricing.preco)

pandas.core.series.Series

In [8]:
pricing['data'] = pd.to_datetime(pricing['data'], format='%d/%m/%Y')

ValueError: time data '30 Jul' does not match format '%d/%m/%Y' (match)

In [ ]:
pricing['carcaça'] = pricing['unidade'].apply(lambda x: 'kg' if ['preco']*100 else '['preco']*1')
    

6. Creat Currency equalizing column

7. Plot histogram of most competitive markets

8. Export to CSV file for further temporal analisys